In [ ]:
library(GenomicRanges)
library(dplyr)
library(data.table)
library(ggplot2)
library(viridis)
library(readxl)
library(pheatmap)
library(RColorBrewer)
library(tidyr)
library(gridExtra)
library(corrplot)
library(ggfortify)
library(ggdendro)
library(umap)
library(DESeq2)
library(rjson)
library(ggrepel)
library(ggpubr)

In [ ]:
rPath = "CART_CUTRUN_Project/results/RNAseq/process/RSEM/"
outPath = "CART_CUTRUN_Project/results/RNAseq/analysis/RSEM/"
figPath = "CART_CUTRUN_Project/results/paper_figure/RNAseq_QC/"
## Read in data
cellList <- c("N", "CM", "EM", "EMRA")
hdList <- paste0("HD", c(1:3, 5:7))
exprList = c("Input", "Product", "Stim1", "Stim2", "Stim3")

# alignD = c()
# for(cell in cellList){
#   for(expr in exprList){
#     for(hd in hdList){

#       rAlignTmp = fread(paste0(rPath, "RNA_CD8_", cell, "_", expr, "_", hd, ".summary"))
#       rAlignExon = rAlignTmp[nrow(rAlignTmp)]%>% as.numeric
#       rAlign = fread(paste0(rPath, "RNA_CD8_", cell, "_", expr, "_", hd, ".stat/RNA_CD8_", cell, "_", expr, "_", hd, ".cnt"), fill = TRUE, header = FALSE)[1:3, ]

#       alignD = data.frame(cell = cell, expr = expr, hd = hd,
#                           seqDepth = as.numeric(rAlign$V4[1]),
#                           alignRate = rAlign$V2[1]/as.numeric(rAlign$V4[1]) * 100,
#                           nonIntronicAlign = rAlign$V2[1]/rAlign$V2[1] * 100,
#                           codingExonAlign = rAlignExon/rAlign$V2[1] * 100,
#                           uniExon = as.numeric(rAlign$V1[2])/rAlign$V2[1] * 100,
#                           multiExon = rAlign$V2[2]/rAlign$V2[1] * 100,
#                           type = "HD") %>% rbind(alignD, .)

#     }
#   }

# }

## patient
## Patient data infusion product of CM-derived cells 
patientRaw = read_xlsx("CART_CUTRUN_Project/report/CART_CUTRUN/analysis/20201028_NHL_CUTandRUN_PatientData.xlsx")
cm_ind = which(patientRaw$`CD8+ T cell isolation` == "CD8TCM")
patient_id_list = patientRaw$Xnumber[cm_ind] %>% tolower
patient_id_list

rPath = "CART_CUTRUN_Project/results/RNAseq/process/NHL_RSEM_gencode_v21/"


# for(sample in paste0("Sample_", patient_id_list, "_IP_CD8-CAR")){
#   if(file.exists(paste0(rPath, sample, "/RSEM.summary"))){
#     print(sample)
#     patient_id = sample %>% gsub("_IP_CD8-CAR", "", .) %>% gsub("Sample_", "", .)
#     rAlignTmp = fread(paste0(rPath, sample, "/RSEM.summary"))
#     rAlignExon = rAlignTmp[nrow(rAlignTmp)]%>% as.numeric
#     rAlign = fread(paste0(rPath, sample, "/RSEM.stat/RSEM.cnt"), fill = TRUE, header = FALSE)[1:3, ]
   
#     alignD = data.frame(cell = "Patient", expr = "Product", hd = patient_id,
#                         seqDepth = as.numeric(rAlign$V4[1]),
#                         alignRate = rAlign$V2[1]/as.numeric(rAlign$V4[1]) * 100,
#                         nonIntronicAlign = rAlign$V2[1]/rAlign$V2[1] * 100,
#                         codingExonAlign = rAlignExon/rAlign$V2[1] * 100,
#                         uniExon = as.numeric(rAlign$V1[2])/rAlign$V2[1] * 100,
#                         multiExon = rAlign$V2[2]/rAlign$V2[1] * 100,
#                         type = "Patient") %>% rbind(alignD, .)

#   }

# }
 

# # hdList <- paste0("HD", 4:7)
# # exprList = c("Input")
# # for(cell in cellList){
# #   for(expr in exprList){
# #     for(hd in hdList){
# # 
# #       kAlign = fromJSON(file = paste0(kPath,  "RNA_CD8_", cell, "_", expr, "_", hd, "/run_info.json"))
# #       rAlignExon = fread(paste0(rPath, "RNA_CD8_", cell, "_", expr, "_", hd, ".summary")) %>% as.numeric
# #       rAlign = fread(paste0(rPath, "RNA_CD8_", cell, "_", expr, "_", hd, ".stat/RNA_CD8_", cell, "_", expr, "_", hd, ".cnt"), fill = TRUE, header = FALSE)[1:3, ]
# # 
# #       alignD = data.frame(cell = cell, expr = expr, hd = hd,
# #                           seqDepth = as.numeric(kAlign$n_processed),
# #                           alignRate = rAlign$V4[1]/as.numeric(kAlign$n_processed) * 100,
# #                           nonIntronicAlign = rAlign$V2[1]/rAlign$V4[1] * 100,
# #                           codingExonAlign = rAlignExon/rAlign$V4[1] * 100,
# #                           uniExon = as.numeric(rAlign$V1[2])/rAlign$V2[1] * 100,
# #                           multiExon = rAlign$V2[2]/rAlign$V2[1] * 100,
# #                           type = "RSEM") %>% rbind(alignD, .)
# # 
# #     }
# #   }
# # 
# # }
# alignD$cell = factor(alignD$cell, levels = c("N", "CM", "EM", "EMRA", "Patient"))


# save(alignD, file = paste0(outPath, "RData/RSEM_alignSummary_HD1-3_5-7_N_CM_EM_EMRA_Patients.RData"))
load(file = paste0(outPath, "RData/RSEM_alignSummary_HD1-3_5-7_N_CM_EM_EMRA_Patients.RData"))


In [ ]:
palette1 <- RColorBrewer::brewer.pal(8, "Set1")
palette2 <- RColorBrewer::brewer.pal(8, "Set2")
# Combine them to create a 16-color palette
palette_col <- c(palette1, palette2)
palette_col
figPath = "CART_CUTRUN_Project/results/paper_figure/RNAseq_QC/"


In [ ]:
# Sequencing depth
alignD %>% filter((expr == "Input" & cell != "EMRA") | (expr == "Product" & cell %in% c("CM", "EM", "Patient"))) %>%
ggplot(aes(x = cell, y = seqDepth, fill = cell)) +
  geom_boxplot(outlier.shape = NA) +
  geom_jitter(aes(color = hd), position = position_jitter(0.15)) + 
  facet_grid(~expr, scales = "free", space = "free_x") +
  scale_fill_viridis(discrete = TRUE, begin = 0.1, option = "magma", alpha = 0.5) +
  scale_color_manual(values = palette_col) +
  theme_bw(base_size = 20) +
  ylab("Sequencing Depth") +
  xlab("") +
  ggpubr::rremove("legend.title") +
  ggpubr::rotate_x_text(angle = 0)

ggsave(paste0(figPath, "RNAseq_SequencingDepth.pdf"), width = 8, height = 8)

# Sequencing depth - HD
alignD %>% filter(hd %in% hdList) %>% filter((expr == "Input" & cell != "EMRA") | (expr == "Product" & cell %in% c("CM", "EM", "Patient"))) %>%
ggplot(aes(x = cell, y = seqDepth, fill = cell)) +
  geom_boxplot(outlier.shape = NA) +
  geom_jitter(aes(color = hd), position = position_jitter(0.15)) + 
  facet_grid(~expr, scales = "free", space = "free_x") +
  scale_fill_viridis(discrete = TRUE, begin = 0.1, end = 0.75, option = "magma", alpha = 0.5) +
  scale_color_manual(values = palette_col) +
  theme_bw(base_size = 20) +
  ylab("Sequencing Depth") +
  xlab("") +
  ggpubr::rremove("legend.title") +
  ggpubr::rotate_x_text(angle = 0)

ggsave(paste0(figPath, "RNAseq_SequencingDepth_HD.pdf"), width = 8, height = 8)

# Sequencing depth - PT
alignD %>% filter(hd %in% patient_id_list) %>% filter((expr == "Input" & cell != "EMRA") | (expr == "Product" & cell %in% c("CM", "EM", "Patient"))) %>%
ggplot(aes(x = cell, y = seqDepth, fill = cell)) +
  geom_boxplot(outlier.shape = NA) +
  geom_jitter(aes(color = hd), position = position_jitter(0.15)) + 
  facet_grid(~expr, scales = "free", space = "free_x") +
  scale_fill_viridis(discrete = TRUE, begin = 1, option = "magma", alpha = 0.5) +
  scale_color_manual(values = palette_col[7:16]) +
  theme_bw(base_size = 20) +
  ylab("Sequencing Depth") +
  xlab("") +
  ggpubr::rremove("legend.title") +
  ggpubr::rotate_x_text(angle = 0)
ggsave(paste0(figPath, "RNAseq_SequencingDepth_PT.pdf"), width = 8, height = 8)

alignD %>% filter(hd %in% patient_id_list) %>% filter((expr == "Input" & cell != "EMRA") | (expr == "Product" & cell %in% c("CM", "EM", "Patient"))) %>%
ggplot(aes(x = hd, y = seqDepth, fill = hd)) +
  geom_bar(stat = "identity", position = position_dodge()) +
  # scale_fill_viridis(discrete = TRUE, begin = 0.1, option = "magma", alpha = 0.8) +
  scale_fill_manual(values = palette_col[7:16]) +
  theme_bw(base_size = 20) +
  ylab("Sequencing Depth") +
  xlab("Patient") +
  ggpubr::rremove("legend") +
  ggpubr::rotate_x_text(angle = 0)

ggsave(paste0(figPath, "RNAseq_SequencingDepth_PT_bar.pdf"), width = 8, height = 8)

In [ ]:
# Alignment rate
alignD  %>% filter((expr == "Input" & cell != "EMRA") | (expr == "Product" & cell %in% c("CM", "EM", "Patient"))) %>% ggplot(aes(x = cell, y = alignRate, fill = cell)) +
  geom_boxplot(outlier.shape = NA) +
  geom_jitter(aes(color = hd), position = position_jitter(0.15)) + 
  facet_grid(~expr, scales = "free", space = "free_x") +
  scale_fill_viridis(discrete = TRUE, begin = 0.1, option = "magma", alpha = 0.5) +
  scale_color_manual(values = palette_col) +
  theme_bw(base_size = 20) +
  ylab("Alignment Rate") +
  xlab("") +
  ggpubr::rremove("legend.title") +
  ggpubr::rotate_x_text(angle = 0)

ggsave(paste0(figPath, "RNAseq_AlignmentRate.pdf"), width = 8, height = 8)

# Alignment rate - HD
alignD %>% filter(hd %in% hdList) %>% filter((expr == "Input" & cell != "EMRA") | (expr == "Product" & cell %in% c("CM", "EM", "Patient"))) %>% ggplot(aes(x = cell, y = alignRate, fill = cell)) +
  geom_boxplot(outlier.shape = NA) +
  geom_jitter(aes(color = hd), position = position_jitter(0.15)) + 
  facet_grid(~expr, scales = "free", space = "free_x") +
  scale_fill_viridis(discrete = TRUE, begin = 0.1, end = 0.75, option = "magma", alpha = 0.5) +
  scale_color_manual(values = palette_col) +
  theme_bw(base_size = 20) +
  ylab("Alignment Rate") +
  xlab("") +
  ggpubr::rremove("legend.title") +
  ggpubr::rotate_x_text(angle = 0)

ggsave(paste0(figPath, "RNAseq_AlignmentRate_HD.pdf"), width = 8, height = 8)

# Alignment rate - PT
alignD  %>% filter(hd %in% patient_id_list) %>% filter((expr == "Input" & cell != "EMRA") | (expr == "Product" & cell %in% c("CM", "EM", "Patient"))) %>% ggplot(aes(x = cell, y = alignRate, fill = cell)) +
  geom_boxplot(outlier.shape = NA) +
  geom_jitter(aes(color = hd), position = position_jitter(0.15)) + 
  facet_grid(~expr, scales = "free", space = "free_x") +
  scale_fill_viridis(discrete = TRUE, begin = 1, option = "magma", alpha = 0.5) +
  scale_color_manual(values = palette_col[7:16]) +
  theme_bw(base_size = 20) +
  ylab("Alignment Rate") +
  xlab("") +
  ggpubr::rremove("legend.title") +
  ggpubr::rotate_x_text(angle = 0)
ggsave(paste0(figPath, "RNAseq_AlignmentRate_PT.pdf"), width = 8, height = 8)
# Alignment rate - PT
alignD  %>% filter(hd %in% patient_id_list) %>% filter((expr == "Input" & cell != "EMRA") | (expr == "Product" & cell %in% c("CM", "EM", "Patient"))) %>% ggplot(aes(x = hd, y = alignRate, fill = hd)) +
  geom_bar(stat = "identity") +
  scale_fill_manual(values = palette_col[7:16]) +
  theme_bw(base_size = 20) +
  ylab("Alignment Rate") +
  xlab("Patient") +
  ggpubr::rremove("legend") +
  ggpubr::rotate_x_text(angle = 0) +
  ylim(0, 100)

ggsave(paste0(figPath, "RNAseq_AlignmentRate_PT_bar.pdf"), width = 8, height = 8)

In [ ]:
# Coding exon rate
alignD  %>% filter((expr == "Input" & cell != "EMRA") | (expr == "Product" & cell %in% c("CM", "EM", "Patient"))) %>% ggplot(aes(x = cell, y = codingExonAlign, fill = cell)) +
  geom_boxplot(outlier.shape = NA) +
  geom_jitter(aes(color = hd), position = position_jitter(0.15)) + 
  facet_grid(~expr, scales = "free", space = "free_x") +
  scale_fill_viridis(discrete = TRUE, begin = 0.1, option = "magma", alpha = 0.5) +
  scale_color_manual(values = palette_col) +
  theme_bw(base_size = 20) +
  ylab("Coding Exon Rate") +
  xlab("") +
  ggpubr::rremove("legend.title") +
  ggpubr::rotate_x_text(angle = 0)
ggsave(paste0(figPath, "RNAseq_ExonRate.pdf"), width = 8, height = 8)

# Coding exon rate - HD
alignD %>% filter(hd %in% hdList) %>% filter((expr == "Input" & cell != "EMRA") | (expr == "Product" & cell %in% c("CM", "EM", "Patient"))) %>% ggplot(aes(x = cell, y = codingExonAlign, fill = cell)) +
  geom_boxplot(outlier.shape = NA) +
  geom_jitter(aes(color = hd), position = position_jitter(0.15)) + 
  facet_grid(~expr, scales = "free", space = "free_x") +
  scale_fill_viridis(discrete = TRUE, begin = 0.1, end = 0.75, option = "magma", alpha = 0.5) +
  scale_color_manual(values = palette_col) +
  theme_bw(base_size = 20) +
  ylab("Coding Exon Rate") +
  xlab("") +
  ggpubr::rremove("legend.title") +
  ggpubr::rotate_x_text(angle = 0)
ggsave(paste0(figPath, "RNAseq_ExonRate_HD.pdf"), width = 8, height = 8)

# Coding exon rate - PT
alignD  %>% filter(hd %in% patient_id_list) %>% filter((expr == "Input" & cell != "EMRA") | (expr == "Product" & cell %in% c("CM", "EM", "Patient"))) %>% ggplot(aes(x = cell, y = codingExonAlign, fill = cell)) +
  geom_boxplot(outlier.shape = NA) +
  geom_jitter(aes(color = hd), position = position_jitter(0.15)) + 
  facet_grid(~expr, scales = "free", space = "free_x") +
  scale_fill_viridis(discrete = TRUE, begin = 1, option = "magma", alpha = 0.5) +
  scale_color_manual(values = palette_col[7:16]) +
  theme_bw(base_size = 20) +
  ylab("Coding Exon Rate") +
  xlab("") +
  ggpubr::rremove("legend.title") +
  ggpubr::rotate_x_text(angle = 0)
ggsave(paste0(figPath, "RNAseq_ExonRate_PT.pdf"), width = 8, height = 8)
alignD  %>% filter(hd %in% patient_id_list) %>% filter((expr == "Input" & cell != "EMRA") | (expr == "Product" & cell %in% c("CM", "EM", "Patient"))) %>% ggplot(aes(x = hd, y = codingExonAlign, fill = hd)) +
  geom_bar(stat = "identity") +
  scale_fill_manual(values = palette_col[7:16]) +
  theme_bw(base_size = 20) +
  ylab("Coding Exon Rate") +
  xlab("Patient") +
  ggpubr::rremove("legend") +
  ggpubr::rotate_x_text(angle = 0) +
  ylim(0, 100)
ggsave(paste0(figPath, "RNAseq_ExonRate_PT_bar.pdf"), width = 8, height = 8)

In [ ]:
# Uni-mapping and Multi-mapping rate 
alignD  %>% filter((expr == "Input" & cell != "EMRA") | (expr == "Product" & cell %in% c("CM", "EM", "Patient"))) %>% ggplot(aes(x = cell, y = uniExon, fill = cell)) +
  geom_boxplot(outlier.shape = NA) +
  geom_jitter(aes(color = hd), position = position_jitter(0.15)) + 
  facet_grid(~expr, scales = "free", space = "free_x") +
  scale_fill_viridis(discrete = TRUE, begin = 0.1, option = "magma", alpha = 0.5) +
  scale_color_manual(values = palette_col) +
  theme_bw(base_size = 20) +
  ylab("Uniquely Alignment Rate") +
  xlab("") +
  ggpubr::rremove("legend.title") +
  ggpubr::rotate_x_text(angle = 0)
ggsave(paste0(figPath, "RNAseq_UniquelyAlignmentRate.pdf"), width = 8, height = 8)

# Uni-mapping and Multi-mapping rate - HD
alignD %>% filter(hd %in% hdList) %>% filter((expr == "Input" & cell != "EMRA") | (expr == "Product" & cell %in% c("CM", "EM", "Patient"))) %>% ggplot(aes(x = cell, y = uniExon, fill = cell)) +
  geom_boxplot(outlier.shape = NA) +
  geom_jitter(aes(color = hd), position = position_jitter(0.15)) + 
  facet_grid(~expr, scales = "free", space = "free_x") +
  scale_fill_viridis(discrete = TRUE, begin = 0.1,  end = 0.75, option = "magma", alpha = 0.5) +
  scale_color_manual(values = palette_col) +
  theme_bw(base_size = 20) +
  ylab("Uniquely Alignment Rate") +
  xlab("") +
  ggpubr::rremove("legend.title") +
  ggpubr::rotate_x_text(angle = 0)
ggsave(paste0(figPath, "RNAseq_UniquelyAlignmentRate_HD.pdf"), width = 8, height = 8)

# Uni-mapping and Multi-mapping rate - PT
alignD  %>% filter(hd %in% patient_id_list) %>% filter((expr == "Input" & cell != "EMRA") | (expr == "Product" & cell %in% c("CM", "EM", "Patient"))) %>% ggplot(aes(x = cell, y = uniExon, fill = cell)) +
  geom_boxplot(outlier.shape = NA) +
  geom_jitter(aes(color = hd), position = position_jitter(0.15)) + 
  facet_grid(~expr, scales = "free", space = "free_x") +
  scale_fill_viridis(discrete = TRUE, begin = 1, option = "magma", alpha = 0.5) +
  scale_color_manual(values = palette_col[7:16]) +
  theme_bw(base_size = 20) +
  ylab("Uniquely Alignment Rate") +
  xlab("") +
  ggpubr::rremove("legend.title") +
  ggpubr::rotate_x_text(angle = 0)
ggsave(paste0(figPath, "RNAseq_UniquelyAlignmentRate_PT.pdf"), width = 8, height = 8)
alignD  %>% filter(hd %in% patient_id_list) %>% filter((expr == "Input" & cell != "EMRA") | (expr == "Product" & cell %in% c("CM", "EM", "Patient"))) %>% ggplot(aes(x = hd, y = uniExon, fill = hd)) +
  geom_bar(stat = "identity") +
  scale_fill_manual(values = palette_col[7:16]) +
  theme_bw(base_size = 20) +
  ylab("Uniquely Alignment Rate") +
  xlab("Patient") +
  ggpubr::rremove("legend") +
  ggpubr::rotate_x_text(angle = 0) +
  ylim(0, 100)
ggsave(paste0(figPath, "RNAseq_UniquelyAlignmentRate_PT_bar.pdf"), width = 8, height = 8)

In [ ]:
## Patient data infusion product of CM-derived cells 
patientRaw = read_xlsx("CART_CUTRUN_Project/report/CART_CUTRUN/analysis/20201028_NHL_CUTandRUN_PatientData.xlsx")
cm_ind = which(patientRaw$`CD8+ T cell isolation` == "CD8TCM")
patient_id_list = patientRaw$Xnumber[cm_ind] %>% tolower
patient_id_list

## Patient data infusion product of CM-derived cells 
inPath = "CART_CUTRUN_Project/results/RNAseq/process/NHL_RSEM_gencode_v21/"

pt_expCount = c()
pt_name_tmp_list = c()
for(sample in paste0("Sample_", patient_id_list, "_IP_CD8-CAR")){
  if(file.exists(paste0(inPath, sample, "/RSEM.genes.results"))){
    print(sample)
    patient_id = sample %>% gsub("Sample_", "", .) %>% gsub("_IP_CD8-CAR", "", .)
    tmp = fread(paste0(inPath, sample, "/RSEM.genes.results")) %>% dplyr::select(expected_count)
    pt_expCount = cbind(pt_expCount, round(tmp))
    pt_name_tmp_list = c(pt_name_tmp_list, paste0("Product_Patient_", patient_id))
  }
}
pt_expCount = data.frame(pt_expCount)
colnames(pt_expCount) = pt_name_tmp_list #paste0(patient_id_list, "_PT")

# geneID = fread(paste0(inPath, sample, "/RSEM.genes.results"))$gene_id
# pt_expCount$gene_name = geneID

head(pt_expCount)
dim(pt_expCount)

## healthy donor data
## Read in RNA-seq data
cellList <- c("CM", "EM") #c("N", "CM", "EM", "EMRA")
hdList <- paste0("HD", c(1:3, 5:7))
exprList = "Product" #c("Input", "Product", "Stim1", "Stim2", "Stim3")

inPath <- "CART_CUTRUN_Project/results/RNAseq/process/RSEM/" #"CART_CUTRUN_Project/results/RNAseq/process/RSEM/"

hd_expCount = c()
colnames_tmp = c()
for(cell in cellList){
  for(expr in exprList){
    for(hd in hdList){
      # print(paste(expr, cell, hd, sep = "_"))
      sample = paste0("RNA_CD8_", cell, "_", expr, "_", hd)
      if(file.exists(paste0(inPath, sample, ".genes.results"))){
        print(sample)
        dataTmp = fread(paste0(inPath, sample, ".genes.results")) %>% dplyr::select(expected_count)
        # dataTmp <- fread(paste0(inPath, "RNA_CD8_", cell, "_", expr, "_", hd, ".genes.results")) %>% dplyr::select(expected_count)
        hd_expCount <- cbind(hd_expCount, round(dataTmp))
        colnames_tmp = c(colnames_tmp, paste0(expr, "_", cell, "_", hd))
      }

    }
  }

}

cellList <- c("N", "CM", "EM") #c("N", "CM", "EM", "EMRA")
hdList <- paste0("HD", c(1:3, 5:7))
exprList = "Input" #c("Input", "Product", "Stim1", "Stim2", "Stim3")

for(cell in cellList){
  for(expr in exprList){
    for(hd in hdList){
      # print(paste(expr, cell, hd, sep = "_"))
      sample = paste0("RNA_CD8_", cell, "_", expr, "_", hd)
      if(file.exists(paste0(inPath, sample, ".genes.results"))){
        print(sample)
        dataTmp = fread(paste0(inPath, sample, ".genes.results")) %>% dplyr::select(expected_count)
        # dataTmp <- fread(paste0(inPath, "RNA_CD8_", cell, "_", expr, "_", hd, ".genes.results")) %>% dplyr::select(expected_count)
        hd_expCount <- cbind(hd_expCount, round(dataTmp))
        colnames_tmp = c(colnames_tmp, paste0(expr, "_", cell, "_", hd))
      }

    }
  }

}


hd_expCount = data.frame(hd_expCount)
colnames(hd_expCount) = colnames_tmp



In [ ]:
# Distribution of Gene Expression.
pt_data = stack(data.frame(pt_expCount)) %>% mutate(label = ind) %>% separate(ind, into = c("Experiment", "CellType", "HD"), sep = "_")
head(pt_data)

hd_data = stack(data.frame(hd_expCount)) %>% mutate(label = ind) %>% separate(ind, into = c("Experiment", "CellType", "HD"), sep = "_")
head(hd_data)


In [ ]:
# Number of genes that have non-zero expression
rbind(hd_data, pt_data) %>% filter(values > 0) %>% group_by(Experiment, CellType, HD) %>% summarize(geneN = n()) %>% ggplot(aes(x = factor(CellType, levels = c("N", "CM", "EM", "EMRA", "Patient")), y = geneN, fill = factor(CellType, levels = c("N", "CM", "EM", "EMRA", "Patient")))) +
  geom_boxplot() +
  geom_jitter(aes(color = HD), position = position_jitter(0.15)) + 
  facet_grid(~Experiment, scales = "free", space = "free_x") +
  scale_fill_viridis(discrete = TRUE, option = "magma", alpha = 0.8, begin = 0.2) +
  scale_color_manual(values = palette_col) +
  theme_bw(base_size = 20) +
  ggpubr::rremove("legend.title") +
  ggpubr::rotate_x_text(angle = 0) +
  xlab("") +
  ylab("Number of Non-zero Expressed Genes")
ggsave(paste0(figPath, "RNAseq_NonZeroGeneNum.pdf"), width = 8, height = 8)

# Number of genes that have non-zero expression - HD
rbind(hd_data, pt_data) %>% filter(values > 0) %>% group_by(Experiment, CellType, HD) %>% summarize(geneN = n())  %>% filter(HD %in% hdList) %>% 
ggplot(aes(x = factor(CellType, levels = c("N", "CM", "EM", "EMRA", "Patient")), y = geneN, fill = factor(CellType, levels = c("N", "CM", "EM", "EMRA", "Patient")))) +
  geom_boxplot() +
  geom_jitter(aes(color = HD), position = position_jitter(0.15)) + 
  facet_grid(~Experiment, scales = "free", space = "free_x") +
  scale_fill_viridis(discrete = TRUE, option = "magma", alpha = 0.8, begin = 0.2, end = 0.8) +
  scale_color_manual(values = palette_col) +
  theme_bw(base_size = 20) +
  ggpubr::rremove("legend.title") +
  ggpubr::rotate_x_text(angle = 0) +
  xlab("") +
  ylab("Number of Non-zero Expressed Genes")
ggsave(paste0(figPath, "RNAseq_NonZeroGeneNum_HD.pdf"), width = 8, height = 8)

# Number of genes that have non-zero expression - PT
rbind(hd_data, pt_data) %>% filter(values > 0) %>% group_by(Experiment, CellType, HD) %>% summarize(geneN = n())  %>% filter(HD %in% patient_id_list) %>% ggplot(aes(x = factor(CellType, levels = c("N", "CM", "EM", "EMRA", "Patient")), y = geneN, fill = factor(CellType, levels = c("N", "CM", "EM", "EMRA", "Patient")))) +
  geom_boxplot() +
  geom_jitter(aes(color = HD), position = position_jitter(0.15)) + 
  facet_grid(~Experiment, scales = "free", space = "free_x") +
  scale_fill_viridis(discrete = TRUE, option = "magma", alpha = 0.8, begin = 1) +
  scale_color_manual(values = palette_col[7:16]) +
  theme_bw(base_size = 20) +
  ggpubr::rremove("legend.title") +
  ggpubr::rotate_x_text(angle = 0) +
  xlab("") +
  ylab("Number of Non-zero Expressed Genes")
ggsave(paste0(figPath, "RNAseq_NonZeroGeneNum_PT.pdf"), width = 8, height = 8)
# Number of genes that have non-zero expression - PT
rbind(hd_data, pt_data) %>% filter(values > 0) %>% group_by(Experiment, CellType, HD) %>% summarize(geneN = n())  %>% filter(HD %in% patient_id_list) %>% ggplot(aes(x = HD, y = geneN, fill = HD)) +
  geom_bar(stat = "identity") +
  scale_fill_manual(values = palette_col[7:16]) +
  theme_bw(base_size = 20) +
  ggpubr::rremove("legend") +
  ggpubr::rotate_x_text(angle = 0) +
  xlab("Patient") +
  ylab("Number of Non-zero Expressed Genes")
ggsave(paste0(figPath, "RNAseq_NonZeroGeneNum_PT_bar.pdf"), width = 8, height = 8)

In [45]:
rbind(hd_data, pt_data) %>% filter(values > 0) %$% values %>% summary

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
      1      10     106    1702     771 3749910 

In [ ]:
# Percentage of genes that have relatively low expression.
# 10 is the first quantile of overall non-zero gene expression.

rbind(hd_data, pt_data) %>% filter(values < 10) %>% group_by(Experiment, CellType, HD) %>% summarize(LowExpressionNum = n()/nrow(pt_expCount) * 100) %>% ggplot(aes(x = factor(CellType, levels = c("N", "CM", "EM", "EMRA", "Patient")), y = LowExpressionNum, fill = factor(CellType, levels = c("N", "CM", "EM", "EMRA", "Patient")))) +
  geom_boxplot() +
  geom_jitter(aes(color = HD), position = position_jitter(0.15)) + 
  facet_grid(~Experiment, scales = "free", space = "free_x") +
  scale_fill_viridis(discrete = TRUE, option = "magma", alpha = 0.8, begin = 0.2) +
  scale_color_manual(values = palette_col) +
  theme_bw(base_size = 20) +
  ggpubr::rremove("legend.title") +
  ggpubr::rotate_x_text(angle = 0) +
  xlab("") +
  ylab("% of Low Expression Genes") +
  ggtitle("GEX <= 1st Quantile of NonZero Genes: 10")
ggsave(paste0(figPath, "RNAseq_LowExpressedGene.pdf"), width = 8, height = 8)

# Percentage of genes that have relatively low expression.
# 10 is the first quantile of overall non-zero gene expression.
# HD
rbind(hd_data, pt_data) %>% filter(values < 10) %>% group_by(Experiment, CellType, HD) %>% summarize(LowExpressionNum = n()/nrow(pt_expCount) * 100)  %>% filter(HD %in% hdList) %>% ggplot(aes(x = factor(CellType, levels = c("N", "CM", "EM", "EMRA", "Patient")), y = LowExpressionNum, fill = factor(CellType, levels = c("N", "CM", "EM", "EMRA", "Patient")))) +
  geom_boxplot() +
  geom_jitter(aes(color = HD), position = position_jitter(0.15)) + 
  facet_grid(~Experiment, scales = "free", space = "free_x") +
  scale_fill_viridis(discrete = TRUE, option = "magma", alpha = 0.8, begin = 0.2, end = 0.8) +
  scale_color_manual(values = palette_col) +
  theme_bw(base_size = 20) +
  ggpubr::rremove("legend.title") +
  ggpubr::rotate_x_text(angle = 0) +
  xlab("") +
  ylab("% of Low Expression Genes") +
  ggtitle("GEX <= 1st Quantile of NonZero Genes: 10")
ggsave(paste0(figPath, "RNAseq_LowExpressedGene_HD.pdf"), width = 8, height = 8)

# Percentage of genes that have relatively low expression.
# 10 is the first quantile of overall non-zero gene expression.
# PT
rbind(hd_data, pt_data) %>% filter(values < 10) %>% group_by(Experiment, CellType, HD) %>% summarize(LowExpressionNum = n()/nrow(pt_expCount) * 100) %>% filter(HD %in% patient_id_list) %>% ggplot(aes(x = factor(CellType, levels = c("N", "CM", "EM", "EMRA", "Patient")), y = LowExpressionNum, fill = factor(CellType, levels = c("N", "CM", "EM", "EMRA", "Patient")))) +
  geom_boxplot() +
  geom_jitter(aes(color = HD), position = position_jitter(0.15)) + 
  facet_grid(~Experiment, scales = "free", space = "free_x") +
  scale_fill_viridis(discrete = TRUE, option = "magma", alpha = 0.8, begin = 1) +
  scale_color_manual(values = palette_col[7:16]) +
  theme_bw(base_size = 20) +
  ggpubr::rremove("legend.title") +
  ggpubr::rotate_x_text(angle = 0) +
  xlab("") +
  ylab("% of Low Expression Genes") +
  ggtitle("GEX <= 1st Quantile of NonZero Genes: 10")
ggsave(paste0(figPath, "RNAseq_LowExpressedGene_PT.pdf"), width = 8, height = 8)
rbind(hd_data, pt_data) %>% filter(values < 10) %>% group_by(Experiment, CellType, HD) %>% summarize(LowExpressionNum = n()/nrow(pt_expCount) * 100) %>% filter(HD %in% patient_id_list) %>% ggplot(aes(x = HD, y = LowExpressionNum, fill = HD)) +
  geom_bar(stat = "identity") +
  scale_fill_manual(values = palette_col[7:16]) +
  theme_bw(base_size = 20) +
  ggpubr::rremove("legend") +
  ggpubr::rotate_x_text(angle = 0) +
  xlab("Patient") +
  ylab("% of Low Expression Genes") +
  ggtitle("GEX <= 1st Quantile of NonZero Genes: 10")

ggsave(paste0(figPath, "RNAseq_LowExpressedGene_PT_bar.pdf"), width = 8, height = 8)

In [ ]:
# Percentage of genes that have relatively high expression.
# 771 is the third quantile of overall non-zero gene expression.

rbind(hd_data, pt_data) %>% filter(values >= 771) %>% group_by(Experiment, CellType, HD) %>% summarize(LowExpressionNum = n()/nrow(pt_expCount) * 100) %>% ggplot(aes(x = factor(CellType, levels = c("N", "CM", "EM", "EMRA", "Patient")), y = LowExpressionNum, fill = factor(CellType, levels = c("N", "CM", "EM", "EMRA", "Patient")))) +
  geom_boxplot() +
  geom_jitter(aes(color = HD), position = position_jitter(0.15)) + 
  facet_grid(~Experiment, scales = "free", space = "free_x") +
  scale_fill_viridis(discrete = TRUE, option = "magma", alpha = 0.8, begin = 0.2) +
  scale_color_manual(values = palette_col) +
  theme_bw(base_size = 20) +
  ggpubr::rremove("legend.title") +
  ggpubr::rotate_x_text(angle = 0) +
  xlab("") +
  ylab("% of High Expression Genes") +
  ggtitle("GEX >= 3rd Quantile of Nonzero Genes: 771")
ggsave(paste0(figPath, "RNAseq_HighExpressedGene.pdf"), width = 8, height = 8)

# Percentage of genes that have relatively high expression.
# 771 is the third quantile of overall non-zero gene expression.
# HD
rbind(hd_data, pt_data) %>% filter(values >= 771) %>% group_by(Experiment, CellType, HD) %>% summarize(LowExpressionNum = n()/nrow(pt_expCount) * 100)  %>% filter(HD %in% hdList) %>% ggplot(aes(x = factor(CellType, levels = c("N", "CM", "EM", "EMRA", "Patient")), y = LowExpressionNum, fill = factor(CellType, levels = c("N", "CM", "EM", "EMRA", "Patient")))) +
  geom_boxplot() +
  geom_jitter(aes(color = HD), position = position_jitter(0.15)) + 
  facet_grid(~Experiment, scales = "free", space = "free_x") +
  scale_fill_viridis(discrete = TRUE, option = "magma", alpha = 0.8, begin = 0.2, end = 0.8) +
  scale_color_manual(values = palette_col) +
  theme_bw(base_size = 20) +
  ggpubr::rremove("legend.title") +
  ggpubr::rotate_x_text(angle = 0) +
  xlab("") +
  ylab("% of High Expression Genes") +
  ggtitle("GEX >= 3rd Quantile of Nonzero Genes: 771")
ggsave(paste0(figPath, "RNAseq_HighExpressedGene_HD.pdf"), width = 8, height = 8)

# Percentage of genes that have relatively high expression.
# 771 is the third quantile of overall non-zero gene expression.
# PT
rbind(hd_data, pt_data) %>% filter(values >= 771) %>% group_by(Experiment, CellType, HD) %>% summarize(LowExpressionNum = n()/nrow(pt_expCount) * 100) %>% filter(HD %in% patient_id_list) %>% ggplot(aes(x = factor(CellType, levels = c("N", "CM", "EM", "EMRA", "Patient")), y = LowExpressionNum, fill = factor(CellType, levels = c("N", "CM", "EM", "EMRA", "Patient")))) +
  geom_boxplot() +
  geom_jitter(aes(color = HD), position = position_jitter(0.15)) + 
  facet_grid(~Experiment, scales = "free", space = "free_x") +
  scale_fill_viridis(discrete = TRUE, option = "magma", alpha = 0.8, begin = 1) +
  scale_color_manual(values = palette_col[7:16]) +
  theme_bw(base_size = 20) +
  ggpubr::rremove("legend.title") +
  ggpubr::rotate_x_text(angle = 0) +
  xlab("") +
  ylab("% of High Expression Genes") +
  ggtitle("GEX >= 3rd Quantile of Nonzero Genes: 771")
ggsave(paste0(figPath, "RNAseq_HighExpressedGene_PT.pdf"), width = 8, height = 8)
rbind(hd_data, pt_data) %>% filter(values >= 771) %>% group_by(Experiment, CellType, HD) %>% summarize(LowExpressionNum = n()/nrow(pt_expCount) * 100) %>% filter(HD %in% patient_id_list) %>% ggplot(aes(x = HD, y = LowExpressionNum, fill = HD)) +
  geom_bar(stat = "identity") +
  scale_fill_manual(values = palette_col[7:16]) +
  theme_bw(base_size = 20) +
  ggpubr::rremove("legend") +
  ggpubr::rotate_x_text(angle = 0) +
  xlab("Patient") +
  ylab("% of High Expression Genes") +
  ggtitle("GEX >= 3rd Quantile of Nonzero Genes: 771")
ggsave(paste0(figPath, "RNAseq_HighExpressedGene_PT_bar.pdf"), width = 8, height = 8)

In [ ]:
## Patient data infusion product of CM-derived cells 
patientRaw = read_xlsx("CART_CUTRUN_Project/report/CART_CUTRUN/analysis/20201028_NHL_CUTandRUN_PatientData.xlsx")
# cm_ind = which(patientRaw$`CD8+ T cell isolation` == "CD8TCM")
patient_id_list = patientRaw$Xnumber %>% tolower
patient_id_list

## Patient data infusion product of CM-derived cells 
inPath = "CART_CUTRUN_Project/results/RNAseq/process/NHL_RSEM_gencode_v21/"

pt_expCount = c()
pt_name_tmp_list = c()
for(sample in paste0("Sample_", patient_id_list, "_IP_CD8-CAR")){
  if(file.exists(paste0(inPath, sample, "/RSEM.genes.results"))){
    print(sample)
    patient_id = sample %>% gsub("Sample_", "", .) %>% gsub("_IP_CD8-CAR", "", .)
    tmp = fread(paste0(inPath, sample, "/RSEM.genes.results")) %>% dplyr::select(expected_count)
    pt_expCount = cbind(pt_expCount, round(tmp))
    pt_name_tmp_list = c(pt_name_tmp_list, paste0("Product_Patient_", patient_id))
  }
}
pt_expCount = data.frame(pt_expCount)
colnames(pt_expCount) = pt_name_tmp_list #paste0(patient_id_list, "_PT")

geneID = fread(paste0(inPath, sample, "/RSEM.genes.results"))$gene_id
rownames(pt_expCount) = geneID

head(pt_expCount)
dim(pt_expCount)

## healthy donor data
## Read in RNA-seq data
cellList <- c("N", "CM", "EM") #c("N", "CM", "EM", "EMRA")
hdList <- paste0("HD", c(1:3, 5:7))
exprList = c("Input", "Product", "Stim1", "Stim2", "Stim3")

inPath <- "CART_CUTRUN_Project/results/RNAseq/process/HD_RSEM_gencode_v21/" #"CART_CUTRUN_Project/results/RNAseq/process/RSEM/"

hd_expCount = c()
colnames_tmp = c()
for(cell in cellList){
  for(expr in exprList){
    for(hd in hdList){
      # print(paste(expr, cell, hd, sep = "_"))
      sample = paste0("RNA_CD8_", cell, "_", expr, "_", hd)
      if(file.exists(paste0(inPath, sample, "/RSEM.genes.results"))){
        print(sample)
        dataTmp = fread(paste0(inPath, sample, "/RSEM.genes.results")) %>% dplyr::select(expected_count)
        # dataTmp <- fread(paste0(inPath, "RNA_CD8_", cell, "_", expr, "_", hd, ".genes.results")) %>% dplyr::select(expected_count)
        hd_expCount <- cbind(hd_expCount, round(dataTmp))
        colnames_tmp = c(colnames_tmp, paste0(expr, "_", cell, "_", hd))
      }

    }
  }

}


hd_expCount = data.frame(hd_expCount)
colnames(hd_expCount) = colnames_tmp

geneID = fread(paste0(inPath, sample, "/RSEM.genes.results"))$gene_id
rownames(hd_expCount) = geneID

head(hd_expCount)
dim(hd_expCount)

dim(pt_expCount)
dim(hd_expCount)
# countMat = inner_join(pt_expCount, hd_expCount, by = "gene_name")
# rownames(countMat) = countMat$gene_name
# countMat = countMat %>% dplyr::select(-gene_name)
# head(countMat)
# dim(countMat)

saveRDS(list(patientRaw, pt_expCount, hd_expCount), file = "CART_CUTRUN_Project/results/RNAseq/analysis/RSEM/RData/RNAseq_patientMeta_patientRawMatrix_hdRawMatrix.rds")
